In [1]:
clientID = 'TTHvGhN8VYCNdie1uB83vg'
clientSecret = '6FCjvM6iIrthyu7DkM7k__z95sra-g'

import praw
import json
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import re
import math
import os

reddit = praw.Reddit(
    client_id= clientID,
    client_secret= clientSecret,
    user_agent='reg',

)

subreddit = reddit.subreddit('appliances')

def get_submission_info(submission):
    title = submission.title
    content = submission.selftext
    comments = [comment.body.replace('\n', ' ') for comment in sorted(submission.comments, key=lambda x: x.score, reverse=True)[:30] if isinstance(comment, praw.models.Comment)]
    kudos = submission.score
    upvotes = submission.ups
    submission_date = submission.created_utc

    content = content.replace('\n', ' ')
    comments_dates = [comment.created_utc for comment in sorted(submission.comments, key=lambda x: x.score, reverse=True)[:30] if isinstance(comment, praw.models.Comment)]

    return {
        'Title': title,
        'Content': content,
        'Top_Comments': comments,
        'Kudos': kudos,
        'Upvotes': upvotes,
        'Submission_Date': submission_date,
        'Comments_Dates': comments_dates
    }

# subreddit.hot(limit=100)
submission_info_list = []

# Make multiple requests until you accumulate 700 submissions
count = 0
while count < 500:
    # Retrieve submissions with a limit of 100 per request
    for submission in subreddit.search("best", limit=100):
        submission_info = get_submission_info(submission)
        submission_info_list.append(submission_info)
        count += 1
        if count >= 700:
            break  # Exit the loop if we have enough submissions

df = pd.DataFrame(submission_info_list)

def column_to_list(csv_file, column_index):
    df = pd.read_csv(csv_file, skiprows=1)
    column_list = df.iloc[:, column_index].tolist()

    column_list.remove("GOOGLE")
    
    return column_list

# need to add to s3 and get the csv file
csv_file = "cleaned_brands.csv" 
column_index = 1 
unique_brands = column_to_list(csv_file, column_index)

unique_categories = [
    "Air Conditioner",
    "Air Fryer",
    "Air Purifier",
    "Bread Maker",
    "Blender",
    "Clothes Steamer",
    "Coffee Machine",
    "Cooktop",
    "Dishwasher",
    "Dryer",
    "Electric Kettle",
    "Espresso Machine",
    "Fan",
    "Food Processor",
    "Griddle",
    "Grill",
    "Hand Mixer",
    "Heater",
    "Humidifier",
    "Ice Cream Maker",
    "Iron",
    "Juicer",
    "Microwave",
    "Oven",
    "Range Hood",
    "Refrigerator",
    "Rice Cooker",
    "Robot Vacuum",
    "Slow Cooker",
    "Smart Doorbell",
    "Smart Lighting",
    "Smart Lock",
    "Smart Thermostat",
    "Sound System",
    "Stand Mixer",
    "Television",
    "Toaster",
    "Vacuum Cleaner",
    "Waffle Maker",
    "Washing Machine",
    "Water Heater",
    "Water Purifier",
    "Washer"
]

############################ Sentiment analysis section ############################

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # Positive, neutral and negative sentiments

## Purpose: Get sentiment analysis
def get_senti_analysis(sentiment_collection, sentence, brand_cat, max_length=512):
    brand = brand_cat[0]

    # Tokenize and truncate the sentence
    modified_sentence = sentence.lower() 
    tokenized_sentence = tokenizer.encode(modified_sentence, add_special_tokens=True, max_length=max_length, truncation=True)

    # Find indices of brand mentions in the tokenized sentence
    brand_indices = [i for i, token in enumerate(tokenized_sentence) if tokenizer.decode([token]) == brand.lower()]

    # Modify the sentence to focus on the specified brand
    for idx in brand_indices:
        tokenized_sentence[idx] = tokenizer.convert_tokens_to_ids("[MASK]")  # Replace brand mentions with [MASK] token

    # Convert tokenized sentence to PyTorch tensor
    input_ids = torch.tensor([tokenized_sentence])
    sentiment = ''

    # Perform sentiment analysis
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1).tolist()

        if predicted_labels[0] == 1:
            sentiment = "positive"
        elif predicted_labels[0] == 0:
            sentiment = "neutral"
        elif predicted_labels[0] == 2:
            sentiment = "negative"

    sentiment_collection[brand_cat][sentiment] += 1

    return sentiment_collection


## Purpose: To get the brand and category of the sentence, then add to the sentiment collection
## Depends on the get_senti_analysis function
def get_brand_and_category(sentiment_collection, brands, sentences, categories):
    for brand in brands:
        # Tokenize the sentence using regular expressions
        sentence_words = re.findall(r'\b\w+\b', sentences.lower())

        if brand.lower() in sentence_words:
            cat = ""

            for category in categories:
                if category.lower() in sentence_words:
                    cat = category
                    if cat == "Washer":
                        cat = "Washing Machine"

            brand = brand.replace(" ", "").lower()
            brand_cat = (brand, cat)
            
            if brand_cat not in sentiment_collection:
                sentiment_collection[brand_cat] = {"positive": 0, "negative": 0, "neutral": 0}
            
            sentiment_collection = get_senti_analysis(sentiment_collection, sentences, brand_cat)

    return sentiment_collection

sentiment_collection = {}

for idx, row in df.iterrows():
    if row['Top_Comments'] != []:
        for comments in row['Top_Comments']:
            get_brand_and_category(sentiment_collection, unique_brands, comments, unique_categories)

    if row['Content'] != "":
        get_brand_and_category(sentiment_collection, unique_brands, row['Content'], unique_categories)
    
    if row['Title'] != "":
        get_brand_and_category(sentiment_collection, unique_brands, row['Title'], unique_categories)


## Purpose: To combine the sentiment scores of the same brand to have a general understanding of which brands are more popular
combined_sentiment_collection = {}
brands_with_categories = set()
for key in sentiment_collection.keys():
    brand, category = key
    if category != '':
        brands_with_categories.add(brand)

for brand, category in sentiment_collection.keys():
    for key in sentiment_collection.keys():
        if key[0] == brand:
            for sentiment in sentiment_collection[key]:
                if (brand, '') not in combined_sentiment_collection:
                    combined_sentiment_collection[(brand, '')] = {}
                combined_sentiment_collection[(brand, '')][sentiment] = combined_sentiment_collection[(brand, '')].get(sentiment, 0) + sentiment_collection[key][sentiment]



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
## Normalizing popularity scores
def min_max_scaling(scores):
    print(scores)
    values = scores.values()
    min_score = min(values)
    max_score = max(values)
    if min_score == max_score:
        # Handle the case where all values are the same
        return {brand: 0.5 for brand in scores}  # Assigning 0.5 as a default scaled value

    scaled_scores = {brand: (score - min_score) / (max_score - min_score) for brand, score in scores.items()}
    return scaled_scores

In [22]:
## Getting populatiry scores
def get_popular_score(sentiment_dictionary, type_check):
    total_sentiments = {brand: sum(sentiments.values()) for brand, sentiments in sentiment_dictionary.items()}

    # Calculate normalized sentiment scores and popularity score for each brand
    popularity_scores = {}
    for brand, sentiments in sentiment_dictionary.items():
        total = total_sentiments[brand]
        positive_score = sentiments['positive'] / total if total > 0 else 0
        negative_score = sentiments['negative'] / total if total > 0 else 0
        neutral_score = sentiments['neutral'] / total if total > 0 else 0
        
        # Penalize brands with fewer counts using logarithmic scaling
        penalization_factor = math.log10(total + 1)  # Adding 1 to prevent division by zero
        popularity_score = ((positive_score * 0.65) + (neutral_score * 0.15) - (negative_score * 0.2)) * penalization_factor
        if type_check == "general":
            popularity_scores[brand[0]] = popularity_score
        else:
            popularity_scores[brand] = popularity_score

    # Sort brands by popularity score
    sorted_brands = sorted(popularity_scores.items(), key=lambda x: x[1], reverse=True)
    return popularity_scores

# Getting hardware zone data
json_file_hardware = './output/23Mar_Tester.json'

def get_brand_and_category_noSentiment(title, desc, categories, brands):
    brand_cat = ""
    for brand in brands:
        # Tokenize the sentence using regular expressions
        title_words = re.findall(r'\b\w+\b', title.lower())
        description_words = re.findall(r'\b\w+\b', desc.lower())

        if brand.lower() in title_words or brand.lower() in description_words:
            cat = ""

            for category in categories:
                if category.lower() in title_words or category.lower() in description_words:
                    cat = category
                    if cat == "Washer":
                        cat = "Washing Machine"

            ##### MIGHT BE ERRONEOUS #####
            brand = brand.replace(" ", "").lower()
            brand_cat = [brand, cat]

    return brand_cat

hardware_df = pd.read_json(json_file_hardware)
df_hardware = hardware_df.drop(columns=['error'])
df_hardware.dropna(inplace=True)
df_hardware = df_hardware.reset_index(drop=True)

hardware_brandCat = []
brand_cat = ""
for idx, row in df_hardware.iterrows():
    brand_cat = get_brand_and_category_noSentiment(row['title'], row['description'], unique_categories, unique_brands)

    if brand_cat not in hardware_brandCat and brand_cat != '':
        hardware_brandCat.append(brand_cat)

## Purpose: To create a dictionary of brand sentiments based on their category
df_legend = {}
found_categories = set()
for brand_cat, sentiments in sentiment_collection.items():
    brand, category = brand_cat
    
    if category == "":
        continue

    df_name = "df_" + category
    found_categories.add(category)
    
    if df_name not in df_legend:
        df_legend[df_name] = {}
    
    if brand not in df_legend[df_name]:
        df_legend[df_name][brand] = {"positive": sentiments["positive"], "neutral": sentiments["neutral"], "negative": sentiments["negative"]}
    else:
        # If brand already exists, sum up sentiments
        df_legend[df_name][brand]["positive"] += sentiments["positive"]
        df_legend[df_name][brand]["neutral"] += sentiments["neutral"]
        df_legend[df_name][brand]["negative"] += sentiments["negative"]

dfs = {}

# Iterate over the dictionary
for key, value in df_legend.items():
    
    final_df = pd.DataFrame(value).transpose()
    dfs[key] = final_df

## Purpose: To get the popularity scores of the brands in each category and scale them
pop_col_list = []
for df_name, internal_df in df_legend.items():
    pop_col = get_popular_score(internal_df, "category")
    df_category = df_name[3:]

    pop_col_list.append([pop_col, df_category])

## Adding in advertisement data from hardware zone
for column, category in pop_col_list:
    for brand, cat in hardware_brandCat:
        if cat == category:
            column[brand] = 1

In [23]:
## Scaling the popularity scores        
scaled_pop_col_list = []
for pop_list in pop_col_list:
    scaled_scores = min_max_scaling(pop_list[0])
    scaled_pop_col_list.append(scaled_scores)

{'GE': -0.05122710528293813, 'MIELE': 0.9884171554996024, 'LG': 0.9237543509342077, 'BOSCH': 0.04346097776309703, 'WHIRLPOOL': 0.8331539230050247, 'SAMSUNG': 0.44151066030717245}
{'GE': 0.39669821758265894, 'LG': 1.2166115334138488, 'WHIRLPOOL': 1.1099206144636586, 'MIELE': 0.7892676968176956, 'SAMSUNG': 0.4446578573620821, 'ELECTROLUX': 0.6338287198795659, 'BOSCH': -0.20322595720552575, 'SMEG': 0.11672268755754654, 'MITSUBISHI': 0.15620890277373375, 'TOWER': -0.15563025007672873}
{'SMEG': 0.6769052453528464, 'BERTAZZONI': 0.6769052453528464, 'BOSCH': 0.4131935296043498, 'ELECTROLUX': 0.5819913249503637, 'GE': 0.13222192947339192, 'LG': 0.41655707406329007, 'SAMSUNG': -0.24082399653118497, 'MIELE': 0.41655707406329007, 'TURBO': 0.11672268755754654}
{'GE': -0.15563025007672873, 'BOSCH': -0.15563025007672873}
{'LG': 1.0115966254987367, 'GE': 0.05659893391883273, 'SAMSUNG': 0.33555638111271135, 'BOSCH': 0.01652774118417397, 'WHIRLPOOL': 0.9197326761810317}
{'WHIRLPOOL': 0.6769052453528464

In [28]:
dfs

{'df_Dishwasher':            positive  neutral  negative  popularity
 GE               10        0        40    0.000000
 MIELE            40        5         0    1.000000
 LG               40        0         5    0.937803
 BOSCH            35        0       100    0.091077
 WHIRLPOOL        45        5        10    0.850657
 SAMSUNG          10        0         5    0.473948,
 'df_Washing Machine':             positive  neutral  negative  popularity
 GE                30       10        30    0.422530
 LG               120       15         5    1.000000
 WHIRLPOOL         50        0         0    0.924857
 MIELE             25       10         0    0.699019
 SAMSUNG           20        0        15    0.456308
 ELECTROLUX        20        5         5    0.589543
 BOSCH              0       10        35    0.000000
 SMEG               0        5         0    0.225342
 MITSUBISHI         0       10         0    0.253152
 TOWER              0        0         5    0.033522,
 'df_Oven': 

In [29]:
## Purpose: To add the popularity scores to the dataframes
for i in range(len(scaled_pop_col_list)):
    target_df = dfs[list(dfs.keys())[i]]
    target_df['popularity'] = pd.Series(scaled_pop_col_list[i], index=target_df.index)

############################ JSON output section ############################

output_directory = "./output/"

# Make sure the directory exists to save the JSON files
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

### Ouptut for json file on general popularity of brands
popular_brands = get_popular_score(combined_sentiment_collection, "general")
scaled_scores = min_max_scaling(popular_brands)

# Convert to JSON
json_data = json.dumps(dict(scaled_scores), indent=4)
outputLine = output_directory + "general_popular_brands.json"

# Write JSON data to a file
with open(outputLine, 'w') as json_file:
    json_file.write(json_data)


# Iterate over the dataframes in the dfs dictionary
for df_name, final_df in dfs.items():
    # Convert DataFrame to JSON
    json_data = final_df.to_json(orient="index")
    
    # Create the filename
    filename = os.path.join(output_directory, f"{df_name}.json")
    
    # Write JSON data to file
    with open(filename, "w") as json_file:
        json_file.write(json_data)


{'WHIRLPOOL': 1.8172361654640476, 'BOSCH': -0.3976073486376901, 'GE': -0.12088028476270986, 'MIELE': 1.355731175637765, 'LG': 2.0484853030134076, 'SAMSUNG': 0.06244099385489667, 'SMEG': 0.8463661165263562, 'BERTAZZONI': 1.048309506867828, 'ELECTROLUX': 1.0730441092090033, 'SHARP': 0.10034333188799374, 'MIDEA': 0.6769052453528464, 'TOWER': -0.29827233876685455, 'PANASONIC': 0.5057983127493684, 'TOSHIBA': 0.8594425415770476, 'MITSUBISHI': 0.15620890277373375, 'PHILIPS': -0.15563025007672873, 'HISENSE': 0.44151066030717245, 'TURBO': 0.11672268755754654, 'TIGER': -0.15563025007672873}


In [30]:
# Iterate over the dataframes in the dfs dictionary
for df_name, final_df in dfs.items():
    # Convert DataFrame to JSON
    json_data = final_df.to_json(orient="index")
    
    # Create the filename
    filename = os.path.join(output_directory, f"{df_name}.json")
    
    # Write JSON data to file
    with open(filename, "w") as json_file:
        json_file.write(json_data)

# convert found_categories to a json file
my_list = list(found_categories)

# Specify the output file path
output_file_path = "found_categories.json"

# Write the JSON data to the output file
with open(output_file_path, "w") as output_file:
    json.dump(my_list, output_file)


In [13]:
dfs

{'df_Dishwasher':            positive  neutral  negative  popularity
 GE               10        0        40    0.000000
 MIELE            40        5         0    1.000000
 LG               40        0         5    0.937803
 BOSCH            35        0       100    0.091077
 WHIRLPOOL        45        5        10    0.850657
 SAMSUNG          10        0         5    0.473948,
 'df_Washing Machine':             positive  neutral  negative  popularity
 GE                30       10        30    0.456860
 LG               120       15         5    0.916292
 WHIRLPOOL         50        0         0    1.000000
 MIELE             25       10         0    0.755813
 SAMSUNG           20        0        15    0.493383
 ELECTROLUX        20        5         5    0.637442
 BOSCH              0       10        35    0.000000
 SMEG               0        5         0    0.243650
 MITSUBISHI         0       10         0    0.273720
 TOWER              0        0         5    0.036246,
 'df_Oven': 

In [33]:
brand = "Washing machine"

brand = brand.replace(" ", "").lower()
print(brand)

washingmachine
